In [18]:
import pickle
import time
from tqdm import tqdm
from bitmex import bitmex
from datetime import timedelta, datetime
from dateutil import parser
import math

In [2]:
bitmex_api_key = '1JnAJmntWdiOeYpv2bCJIGCj'    #Testnet API keys
bitmex_api_secret = '5TE3OhqDTCl6ffrCCug-XwA0mINL-1tSiBbctXIfozHqFkYX'

In [3]:
binsizes = {"1m": 1, "5m": 5, "1h": 60, "1d": 1440}
batch_size = 500
bitmex_client = bitmex(test=True, api_key=bitmex_api_key, api_secret=bitmex_api_secret) #Fetching from bitmex testnet

/home/ranvir/anaconda3/lib/python3.8/site-packages/swagger_spec_validator/validator20.py:48: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  warnings.warn(


In [5]:
default_symbol = "LINKUSD"
default_kline_size = "1h"

In [39]:
temp = bitmex_client.Trade.Trade_getBucketed(symbol=default_symbol, binSize=default_kline_size, count=3, reverse=True).result()[0]
print(len(temp))

3


In [8]:
start = bitmex_client.Trade.Trade_getBucketed(symbol=default_symbol, binSize=default_kline_size, count=1, reverse=False).result()[0][0]['timestamp']
end = bitmex_client.Trade.Trade_getBucketed(symbol=default_symbol, binSize=default_kline_size, count=1, reverse=True).result()[0][0]['timestamp']
print(start,end)

<class 'datetime.datetime'> 2021-12-09 22:00:00+00:00


In [43]:
minute_timespan = (end - start).total_seconds()/60
num_datapoints = math.ceil(minute_timespan/binsizes[default_kline_size])
num_queries = math.ceil(num_datapoints/batch_size)
close_list = []
open_list = []
timestamp_list = []
print(min_timespan,num_datapoints, num_queries)

80280.0 1338 3


In [44]:
for query_iter in tqdm(range(num_queries)):
    new_start = start + timedelta(minutes = query_iter * batch_size * binsizes[default_kline_size])
    data = bitmex_client.Trade.Trade_getBucketed(symbol=default_symbol, binSize=default_kline_size, count=batch_size, startTime = new_start).result()[0]
    for i in range(len(data)):
        close_list.append(data[i]['close'])
        open_list.append(data[i]['open'])
        timestamp_list.append(data[i]['timestamp'])


100%|██████████| 3/3 [00:00<00:00,  4.82it/s]


In [46]:
stored_vars={}
stored_vars["timestamp_list"] = timestamp_list
stored_vars["close_list"] = close_list
stored_vars["open_list"] = open_list

with open('data/bitmex_LINKUSD_list.pickle','wb') as handle:
    pickle.dump(stored_vars, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [47]:
print(stored_vars)

{'timestamp_list': [datetime.datetime(2021, 10, 15, 4, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 5, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 6, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 7, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 8, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 9, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 10, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 11, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 12, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 13, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 14, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 15, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 16, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 17, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 18, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 19, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 20, 0, tzinfo=tzutc()), datetime.datetime(2021, 10, 15, 21